In [6]:
import sys
import re

START_PATTERN = re.compile(r'^(.*?)<B_ENAMEX$', re.I)
END_SINGLE_PATTERN = re.compile(r'^TYPE="(.*?)">(.*?)<E_ENAMEX>(.*?)$', re.I)
TYPE_PATTERN = re.compile(r'^TYPE="(.*?)">(.*?)$', re.I)
END_MULTI_PATTERN = re.compile(r'^(.*?)<E_ENAMEX>(.*?)$', re.I)
EOS_PATTERN = re.compile(r'^([^<>]*)\.?\t(\d+)$', re.I)
NON_ENTITY_TYPE = 'O'


def check_and_process_eos(token):
    match = re.match(EOS_PATTERN, token)
    if match:
        out.write(match.group(1) + '\t' + cur_type + '\n')
        out.write('.' + '\t' + cur_type + '\n')
        out.write('\n')
        return True
    return False

def convert(infile, outfile, cur_type):
    with open(infile, "r", encoding="utf-8", errors="ignore") as f, open(outfile, 'w', encoding="utf-8") as out:
        for line in f:
            line = line.decode()
            for token in line.strip().split(' '):
                token = token.strip()
                if not token:
                    continue

                match = re.match(START_PATTERN, token)
                if match:
                    if match.group(1):
                        out.write(match.group(1) + '\t' + NON_ENTITY_TYPE + '\n')
                    continue

                match = re.match(END_SINGLE_PATTERN, token)
                if match:
                    out.write(match.group(2) + '\t' + match.group(1) + '\n')
                    cur_type = NON_ENTITY_TYPE
                    if not check_and_process_eos(match.group(3)):
                        out.write(match.group(3) + '\t' + cur_type + '\n')
                    continue

                match = re.match(TYPE_PATTERN, token)
                if match:
                    cur_type = match.group(1)
                    out.write(match.group(2) + '\t' + cur_type + '\n')
                    continue

                match = re.match(END_MULTI_PATTERN, token)
                if match:
                    out.write(match.group(1) + '\t' + cur_type + '\n')
                    cur_type = NON_ENTITY_TYPE
                    if not check_and_process_eos(match.group(2)):
                        out.write(match.group(2) + '\t' + cur_type + '\n')
                    continue

                if check_and_process_eos(token):
                    continue

                out.write(token + '\t' + cur_type + '\n')
            out.write('\n')


infile  = "../Data/NER_data/WFS7.MUClabeled"
outfile = "../Data/NER_data/BIO_NETest.txt"
cur_type = NON_ENTITY_TYPE

In [7]:
convert(infile, "../Data/NER_data/BIO_Test.txt", cur_type)

AttributeError: 'str' object has no attribute 'decode'

In [5]:
convert("../Data/NER_data/Train7.MUClabeled", "../Data/NER_data/BIO_Train.txt", cur_type)